# One-way ANOVA test and pairwise comparisons with TukeyHSD confidence intervals

In [ ]:
# imports and setting matplotlib style
import numpy as np
import scipy
import scipy.stats
import matplotlib.pyplot as plt
from statsmodels.stats.multicomp import pairwise_tukeyhsd
plt.style.use('ggplot')
#print(plt.style.available)


In [ ]:
#dummy data, to be replaced with real data
keys = np.random.choice((0, 1, 2, 3), size=50)
data = np.random.normal(2, 1, size=50)

In [ ]:
# seperate the data according to keys into a dictionary
data = np.fromiter(data, np.float)
data_sep = {}
for k,v in zip(keys, data): data_sep.setdefault(k,[]).append(v)

In [ ]:
# histogram of data for covenience
n, bins, patches = plt.hist(data_sep.values(), int(sum(map(len, data_sep.values()))/4), histtype='stepfilled', alpha=0.75, label=(str(lab) for lab in data_sep.keys()))
plt.legend()
plt.savefig("histograms.png")
plt.show()

In [ ]:
# box plots of data for covenience
plt.boxplot(data_sep.values())
plt.violinplot(data_sep.values())
plt.savefig("boxplots.png")
plt.show()

# Results of anova and pairwise Tukey HSD

In [ ]:
anova_results = scipy.stats.f_oneway(*data_sep.values())
print anova_results[0], anova_results[1]

In [ ]:
results = pairwise_tukeyhsd(data, keys)
print results
